In [ ]:
pip install vaderSentiment


In [ ]:
import gzip
import json
file_path = '/Users/tammychen/Downloads/AMAZON_FASHION_5.json.gz'

with gzip.open(file_path, 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            data = json.loads(line)
            print(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

for sentence in data['reviewText']:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<60} {}".format(sentence, str(vs)))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

file_path = '/Users/tammychen/Downloads/AMAZON_FASHION_5.json.gz'

df = pd.read_json(file_path, lines=True)
df = df[['overall', 'reviewText']]
df['reviewText'] = df['reviewText'].fillna('')

# Assume that ratings above 3 are positive and below or equal to 3 are negative
df['label'] = df['overall'].apply(lambda x: 'positive' if x > 3 else 'negative')

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], df['label'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
